In [98]:
import pandas as pd
import numpy as np

In [ ]:
user_df = pd.read_csv("users.csv",sep=';',error_bad_lines=False,encoding='latin-1')
book_df = pd.read_csv("books.csv",sep=';',error_bad_lines=False,encoding='latin-1')
rating_df = pd.read_csv("ratings.csv",sep=';',error_bad_lines=False,encoding='latin-1')
book_df.columns = ['ISBN','book_title','book_author', 'yearPublication', 'publisher','imageURLS','imageURLM','imageURLL']
user_df.columns = ['userID','location','age']
rating_df.columns = ['userID','ISBN','book_ratings']

In [100]:
user_df.head()

,userID,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [101]:
book_df.head()

,ISBN,book_title,book_author,yearPublication,publisher,imageURLS,imageURLM,imageURLL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [102]:
rating_df.head()

,userID,ISBN,book_ratings
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [103]:
#Check how many unique Books are there
print("Totoal Unique Books in the Dataset - ",len(book_df['book_title'].unique()))

Totoal Unique Books in the Dataset -  242135


In [104]:
# Check how many unique users are there
print("Unique Users in the Dataset -", len(user_df['userID'].unique()))

Unique Users in the Dataset - 278858


In [105]:
# Check how many unique users give ratings to the books
print("Unique Users Give Ratings to Books -")
rating_df['userID'].value_counts()

Unique Users Give Ratings to Books -


11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: userID, Length: 105283, dtype: int64

In [106]:
book_df['book_title'] = book_df['book_title'].apply(lambda x: x.strip())

In [107]:
# Let's combine the Book Dataframe with Rating Dataframe
bookRating = pd.merge(book_df, rating_df, on='ISBN')

In [108]:
poster = bookRating[['ISBN','book_title','imageURLM']]

In [109]:
bookRating = bookRating.drop(['book_author', 'yearPublication', 'publisher','imageURLS','imageURLM','imageURLL'], axis=1)
bookRating.head()

,ISBN,book_title,userID,book_ratings
0,0195153448,Classical Mythology,2,0
1,0002005018,Clara Callan,8,5
2,0002005018,Clara Callan,11400,0
3,0002005018,Clara Callan,11676,8
4,0002005018,Clara Callan,41385,0


In [110]:
bookRating.isnull().sum()

ISBN            0
book_title      0
userID          0
book_ratings    0
dtype: int64

In [111]:
combineRating = (bookRating.groupby(by=['book_title'])['book_ratings'].count().reset_index().rename(columns={'book_ratings':'totalRating'})[['book_title','totalRating']])
combineRating['book_title'].values

array(['!%@ (A Nutshell handbook)', '!Arriba! Comunicacion y cultura',
       '!Trato hecho!: Spanish for Real Life, Combined Edition', ...,
       'Ã?Â?sterlich leben.', 'Ã?Â?stlich der Berge.',
       'Ã?Â?thique en toc'], dtype=object)

In [112]:
final_df = bookRating.merge(combineRating, left_on = 'book_title',right_on='book_title')
final_df.head()

,ISBN,book_title,userID,book_ratings,totalRating
0,0195153448,Classical Mythology,2,0,2
1,0801319536,Classical Mythology,269782,7,2
2,0002005018,Clara Callan,8,5,14
3,0002005018,Clara Callan,11400,0,14
4,0002005018,Clara Callan,11676,8,14


In [113]:
final_df.isnull().sum()

ISBN            0
book_title      0
userID          0
book_ratings    0
totalRating     0
dtype: int64

In [114]:
print(combineRating['totalRating'].describe())

count    241067.000000
mean          4.277383
std          16.738827
min           1.000000
25%           1.000000
50%           1.000000
75%           3.000000
max        2502.000000
Name: totalRating, dtype: float64


In [115]:
print(combineRating['totalRating'].quantile(np.arange(0.9,1,0.01)))

0.90     7.0
0.91     8.0
0.92     9.0
0.93    10.0
0.94    11.0
0.95    13.0
0.96    16.0
0.97    20.0
0.98    29.0
0.99    50.0
Name: totalRating, dtype: float64


In [116]:
threshold = 50
final_df = final_df.query('totalRating >= @threshold')
final_df.head()

,ISBN,book_title,userID,book_ratings,totalRating
31,0399135782,The Kitchen God's Wife,8,0,311
32,0399135782,The Kitchen God's Wife,11676,9,311
33,0399135782,The Kitchen God's Wife,29526,9,311
34,0399135782,The Kitchen God's Wife,36836,0,311
35,0399135782,The Kitchen God's Wife,46398,9,311


In [117]:
final_df.drop_duplicates(['userID','book_title'], inplace=True)

In [118]:
pivot = final_df.pivot(index='book_title',columns='userID',values='book_ratings').fillna(0)

In [119]:
searchable = pivot.copy(deep=True)
searchable['count'] = range(0,len(searchable))

In [120]:
from scipy.sparse import csr_matrix
pivot_df = csr_matrix(pivot.values)

In [121]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(pivot_df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [122]:
query_index = np.random.choice(pivot.shape[0])
distance, indices = model.kneighbors(pivot.iloc[query_index,:].values.reshape(1,-1), n_neighbors=5)

In [124]:
def recommend(book):
  qindex = searchable[searchable.index == book]['count'][0]
  distance, indices = model.kneighbors(pivot.iloc[qindex,:].values.reshape(1,-1), n_neighbors=5)
  books = []
  for i in range(0, len(distance.flatten())):
    if i == 0:
      print("Recommendation For {0}: \n".format(pivot.index[qindex]))
    else:
      print('Recommendation For {0}:{1}, with distance of {2}'.format(i, pivot.index[indices.flatten()[i]], distance.flatten()[i]))
      books.append(pivot.index[indices.flatten()[i]])
  return books

In [125]:
books = recommend("Abduction")

Recommendation For Abduction: 

Recommendation For 1:Contagion, with distance of 0.8743130566783472
Recommendation For 2:Palomino, with distance of 0.8976691191479669
Recommendation For 3:Hide &amp; Seek, with distance of 0.9051817513757328
Recommendation For 4:Toxin, with distance of 0.9100086296180445


In [127]:
import pickle
pickle.dump(searchable,open('booklist.pkl','wb'))
pickle.dump(model,open('model.pkl','wb'))
pickle.dump(pivot,open('pivot.pkl','wb'))

In [128]:
pickle.dump(poster,open('poster.pkl','wb'))